# Q10:
Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [1]:
# Import necessary module: sqlite and pands

import sqlite3

import pandas as pd


# Create connection, con, with context manager, using local instance of SQL lite
with sqlite3.connect(r'C:\Users\fabby\OneDrive\Springboard\SQLFiles Tier 2\sqlite_db_pythonsqlite.db') as con:
    # Import results of a query into pandas dataframe. The query finds the total revenue (from July 2012 through September 2012)
    # of each facility in the dataframe (accounting for guest and member usage separately).
    df = pd.read_sql_query("SELECT info.name as Facility_Name, info.Facility_Revenue\
                            FROM\
                            (SELECT\
                                name,\
                                CASE WHEN b.memid = 0 THEN SUM(b.slots* f.guestcost)\
                                ELSE SUM(b.slots * f.membercost) END AS Facility_Revenue\
                            FROM Facilities as f\
                               LEFT JOIN Bookings AS b\
                                   ON f.facid = b.facid\
                               LEFT JOIN Members AS m\
                                   ON b.memid = m.memid\
                            GROUP BY f.name\
                            ORDER BY Facility_Revenue DESC) AS info", con)



In [2]:
print(df)

     Facility_Name  Facility_Revenue
0   Massage Room 1          112320.0
1   Tennis Court 1           33000.0
2   Tennis Court 2           31950.0
3     Squash Court           19320.0
4  Badminton Court           18739.5
5   Massage Room 2           18240.0
6       Pool Table            4550.0
7    Snooker Table            4540.0
8     Table Tennis            4150.0


df shows that the maximum revenue comes from Massage Room 1 while the minimum revenue came from the Table Tennis facility.

In [3]:
# Follow same procedure as above but filter for where revenue is less than 1000
with sqlite3.connect(r'C:\Users\fabby\OneDrive\Springboard\SQLFiles Tier 2\sqlite_db_pythonsqlite.db') as con:
    df1 = pd.read_sql_query("SELECT info.name as Facility_Name, info.Facility_Revenue\
                            FROM\
                            (SELECT\
                                name,\
                                CASE WHEN b.memid = 0 THEN SUM(b.slots* f.guestcost)\
                                ELSE SUM(b.slots * f.membercost) END AS Facility_Revenue\
                            FROM Facilities as f\
                               LEFT JOIN Bookings AS b\
                                   ON f.facid = b.facid\
                               LEFT JOIN Members AS m\
                                   ON b.memid = m.memid\
                            GROUP BY f.name\
                            ORDER BY Facility_Revenue DESC) AS info\
                            WHERE Facility_Revenue < 1000", con)

In [4]:
print(df1)

Empty DataFrame
Columns: [Facility_Name, Facility_Revenue]
Index: []


df1 shows that none of the facilities generated revenue under 1000 over the time period in the country club database.

# Q11: 
Produce a report of members and who recommended them in alphabetic surname,firstname order.

In [5]:
# Create connection, con, with context manager, using local instance of SQL lite
with sqlite3.connect(r'C:\Users\fabby\OneDrive\Springboard\SQLFiles Tier 2\sqlite_db_pythonsqlite.db') as con:
    # Import a query into pandas dataframe. The query finds a list of members in alphabetical order and who recommended them.
    df2 = pd.read_sql_query("SELECT m1.surname || ', ' || m1.firstname AS Member_Name,\
                                CASE WHEN m1.recommendedby = 0 THEN ''\
                                WHEN m1.recommendedby IS NULL THEN ''\
                                ELSE m2.surname || ', ' || m2.firstname END AS Recommend_By\
                            FROM Members AS m1\
                                LEFT JOIN Members as m2\
                                    ON m1.recommendedby = m2.memid\
                            ORDER BY Member_Name", con)

In [6]:
print(df2)

                 Member_Name        Recommend_By
0            Bader, Florence    Stibbons, Ponder
1                Baker, Anne    Stibbons, Ponder
2             Baker, Timothy     Farrell, Jemima
3                Boothe, Tim         Rownam, Tim
4            Butters, Gerald       Smith, Darren
5               Coplin, Joan      Baker, Timothy
6             Crumpet, Erica        Smith, Tracy
7                Dare, Nancy    Joplette, Janice
8             Farrell, David                None
9            Farrell, Jemima                None
10              GUEST, GUEST                None
11          Genting, Matthew     Butters, Gerald
12                Hunt, John  Purview, Millicent
13              Jones, David    Joplette, Janice
14            Jones, Douglas        Jones, David
15          Joplette, Janice       Smith, Darren
16           Mackenzie, Anna       Smith, Darren
17             Owen, Charles       Smith, Darren
18             Pinker, David     Farrell, Jemima
19        Purview, M

# Q12: 
Find the facilities with their usage by member, but not guests 

In [7]:
# Create connection, con, with context manager, using local instance of SQL lite
with sqlite3.connect(r'C:\Users\fabby\OneDrive\Springboard\SQLFiles Tier 2\sqlite_db_pythonsqlite.db') as con:
    # Import a query into pandas dataframe. The query finds usage of facilities by member.
    df3 = pd.read_sql_query("SELECT\
                                f.name AS Facility_Name,\
                                m.firstname || ', ' || m.surname AS Member_Name,\
                                COUNT(b.bookid) AS Number_of_Bookings\
                            FROM `Bookings` as b\
                                LEFT JOIN `Members` AS m\
                                    ON b.memid = m.memid\
                                LEFT JOIN `Facilities` AS f\
                                    ON b.facid = f.facid\
                            WHERE m.memid > 0\
                            GROUP BY Facility_Name, Member_Name\
                            ORDER by Facility_Name, Number_of_Bookings", con)

In [8]:
print(df3)

       Facility_Name           Member_Name  Number_of_Bookings
0    Badminton Court  Hyacinth, Tupperware                   1
1    Badminton Court         Burton, Tracy                   2
2    Badminton Court        Douglas, Jones                   2
3    Badminton Court        Erica, Crumpet                   2
4    Badminton Court            John, Hunt                   2
..               ...                   ...                 ...
197   Tennis Court 2          David, Jones                  30
198   Tennis Court 2      Ponder, Stibbons                  31
199   Tennis Court 2           Anne, Baker                  35
200   Tennis Court 2         Charles, Owen                  41
201   Tennis Court 2           Tim, Boothe                  52

[202 rows x 3 columns]


# Q13: 
Find the facilities usage by month, but not guests.

In [9]:
# Create connection, con, with context manager, using local instance of SQL lite
with sqlite3.connect(r'C:\Users\fabby\OneDrive\Springboard\SQLFiles Tier 2\sqlite_db_pythonsqlite.db') as con:
    # Import a query into pandas dataframe. The query finds usage of facilities by month.
    df4 = pd.read_sql_query("SELECT\
                                f.name AS Facility_Name,\
                                strftime('%m', starttime) AS Month,\
                                COUNT(b.bookid) AS Number_of_Bookings\
                            FROM `Bookings` AS b\
                                LEFT JOIN Members AS m\
                                    ON b.memid = m.memid\
                                LEFT JOIN `Facilities` AS f\
                                    ON b.facid = f.facid\
                            WHERE m.memid > 0\
                            GROUP BY Facility_Name, Month\
                            ORDER by Facility_Name, Month", con)

In [10]:
print(df4)

      Facility_Name Month  Number_of_Bookings
0   Badminton Court    07                  51
1   Badminton Court    08                 132
2   Badminton Court    09                 161
3    Massage Room 1    07                  77
4    Massage Room 1    08                 153
5    Massage Room 1    09                 191
6    Massage Room 2    07                   4
7    Massage Room 2    08                   9
8    Massage Room 2    09                  14
9        Pool Table    07                 103
10       Pool Table    08                 272
11       Pool Table    09                 408
12    Snooker Table    07                  68
13    Snooker Table    08                 154
14    Snooker Table    09                 199
15     Squash Court    07                  23
16     Squash Court    08                  85
17     Squash Court    09                  87
18     Table Tennis    07                  48
19     Table Tennis    08                 143
20     Table Tennis    09         